In [ ]:
# Import Required Libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
## PREPROCESSING
# Load DataFrame
df = pd.read_excel('./AmazonSaleReport.xlsx') 
df_timeseries = df[['Date', 'Amount']] 

# Transform date into datetime object
df_timeseries['Date'] = pd.to_datetime(df_timeseries['Date'])

# Set Date as Index
df_timeseries.set_index('Date', inplace=True)
df_timeseries.head()  # Display the first few rows of the dataframe with the new index
df.sort_index(inplace=True) 

# Resample DataFrame, I am applying a rolling window of 1 day, smoothing the data
df_resampled = df_timeseries.resample('D').sum()  # Resample by day and sum the 'Amount' column
df_resampled.head()  # Display the first few rows of the resampled dataframe

# Plot the resampled data
plt.figure(figsize=(10, 5))
plt.plot(df_resampled.index, df_resampled['Amount'], marker='o')
plt.title('Daily Amount')
plt.xlabel('Date')
plt.ylabel('Amount')
plt.grid(True)
plt.show()

In [ ]:
# possibility to implement various rolling windows and then compare the results

In [ ]:
# Split the data into training and testing sets
# 80% for training, 20% for testing
# We do not shuffle the data because the order is important in time series
train_size = int(len(df_resampled) * 0.8)  # 80% for training, 20% for testing
train, test = df_resampled.iloc[:train_size], df_resampled.iloc[train_size:]

# Plot the train vs the test data
plt.figure(figsize=(10, 5))
plt.plot(train.index, train['Amount'], label='Train')
plt.plot(test.index, test['Amount'], label='Test')
plt.title('Daily Amount')
plt.xlabel('Date')
plt.ylabel('Amount')
plt.grid(True)
plt.legend()
plt.show()

# CHECKING STATIONARITY

In [ ]:
# ADF TEST

In [ ]:
# KPSS TEST

In [ ]:
# Check for normality

# CHECKING CORRELATION

In [ ]:
# ACF plot

In [ ]:
# PACF plot